In [1]:
import pandas as pd 
import numpy as np 
import matplotlib as mpl 
# 导入数据分析库
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from  sklearn.preprocessing import  MinMaxScaler,OneHotEncoder
from sklearn.metrics import silhouette_score  # 导入轮廓系数计算模块
from sklearn.cluster import KMeans  # KMeans模块

#Jupyter 魔法函数，可以显示绘图
%matplotlib inline 

#忽略警告
import warnings 
warnings.filterwarnings("ignore")

## 设置属性防止中文乱码
mpl.rcParams['font.sans-serif'] = [u'Songti SC']
mpl.rcParams['axes.unicode_minus'] = False

# 读取数据
raw_data = pd.read_table('./ad_performance_cn.csv',delimiter = '\,')
# raw_data = pd.read_csv('./ad_performance_cn.csv')
# 预览数据
raw_data.head()

In [ ]:
# 查看字段数据类型
raw_data = raw_data.drop(['Unnamed: 0'],axis = 1)
print('字段数据类型\n',raw_data.dtypes)
# 查看描述统计分析
print('\n数据描述统计信息\n',raw_data.describe().round(2).T)

In [ ]:
# 查看是否有缺失值
na_cols = raw_data.isnull().any(axis = 0) # 查看每一列是否有缺失值
na_name = na_cols[na_cols > 0] # 查看具有缺失值的列
na_sum = raw_data.isnull().any(axis = 1).sum() #查看一共有多少数据缺失
print('含有缺失值的列：',na_name,'\n缺失的数据量',na_sum)

In [ ]:
#（3）缺失值可视化
import missingno as msno
msno.bar(raw_data)#查看数据集数据完整性

In [ ]:
#（4）提取缺失值信息
raw_data[raw_data.isnull().values==True]

In [ ]:
#（5）缺失值填充&删除
raw_data = raw_data.fillna(419.77)
# 再次查看是否有缺失值
print(raw_data[raw_data.isna().values == True])

# 相关性分析
raw_data[raw_data.describe().columns].corr().round(2)
print (raw_data[raw_data.describe().columns].corr().round(2).T)
# df_1.columns=df_1.describe().columns
# df_1.corr().round(2)

import seaborn as sns 
print 
corr = raw_data[raw_data.describe().columns].corr().round(2)
# sns.heatmap(corr,cmap='Reds',annot = True)

f,ax = plt.subplots(figsize=(20, 10))
ax = sns.heatmap(corr,cmap='Reds',annot = True)


In [ ]:
# (1)剔除缺失值
#平均停留时间有缺失值，同时由于“访问深度”和“平均停留时间”相关性比较高，选择删除“平均停留时间”列
raw_data2 = raw_data.drop(['平均停留时间'],axis = 1)

# (2)对非数值变量进行one-hot编码，转化为数值变量
cols = ['素材类型','广告类型','合作方式','广告尺寸','广告卖点']
one_matrix = pd.get_dummies(raw_data2[cols])
one_matrix.head()

In [ ]:
# (3)数据标准化
data_matrix = raw_data2.iloc[:,2:7] # 第一列为渠道编号，不是数值，无法标准化，应去掉
model_norm = MinMaxScaler() # 建立MinMaxScaler模型对象，将数据归一到 [ 0，1 ] 
data_norm = model_norm.fit_transform(data_matrix) #标准化处理

# 合并归一化数据和one-hot编码数据
X = np.hstack((data_norm,one_matrix))

# 预览数据
print(X.round(2))

# 建立模型

In [ ]:
'''
基于最优的数据尺度确定K-Means算法中的K值,引入轮廓系数评估聚类质量。
轮廓系数的思想：
最佳的聚类类别划分从数据特征上看，类内距离最小化且类别间距离最大化，
直观的理解就是“物以类聚”：同类的“聚集”“抱团”，不同类的离得远。
轮廓系数通过枚举每个K计算平均轮廓系数得到最佳值。
轮廓系数的值介于 [-1,1] ，越趋近于1代表内聚度和分离度都相对较优。
将所有点的轮廓系数求平均，就是该聚类结果总的轮廓系数。
'''
# 通过平均轮廓系数检验得到最佳KMeans聚类模型
score_list = list() #建立列表存储每个K下模型的平均轮廓系数
silhouette_int = -1 #初始化的平均轮廓系数阈值
for n_clusters in range(2,8): #遍历2到8有限个组
    model_kmeans = KMeans(n_clusters = n_clusters) #建立聚类模型对象
    labels_tmp = model_kmeans.fit_predict(X) #训练聚类模型
    silhouette_tmp = silhouette_score(X,labels_tmp)#计算每个K下的平均轮廓系数
    if silhouette_tmp > silhouette_int: #如果平均轮廓系数更高
        best_k = n_clusters #保存最好的k值
        silhouette_int = silhouette_tmp #保存平均轮廓系数得分
        best_kmeans = model_kmeans # 保存模型实例对象
        cluster_labels_k = labels_tmp #保存聚类标签
    score_list.append([n_clusters,silhouette_tmp]) # 将每次K及其得分追加到列表
print('K值对应的轮廓系数为：\n',np.array(score_list))
print('最优的K值为：\n',best_k,'对应的轮廓系数为：\n',silhouette_int) 

# 聚类结果特征分析

In [ ]:
# 将原始数据与聚类标签整合
cluster_labels = pd.DataFrame(cluster_labels_k,columns = ['clusters']) # 获得训练集下的标签信息
merge_data = pd.concat((raw_data2,cluster_labels),axis = 1) # 将原始处理过的数据跟聚类标签整合
merge_data.head()

In [ ]:
# 计算每个聚类下样本数量和占比情况
#计算每个聚类类别的样本量
clusters_count = pd.DataFrame(merge_data['渠道代号'].groupby(merge_data['clusters']).count()).T.rename({'渠道代号':'counts'})
#计算每个聚类类别的样本占比
clusters_rat = (clusters_count/ len(merge_data)).round(2).rename({'counts':'percentage'})
print('每个聚类下的样本量：\n',clusters_count,'\n\n每个聚类下的样本占比：\n',clusters_rat)

# print ('aaaaaaa')
# print (clusters_count,'---clusters_count')
# print (clusters_count.columns,'----clusters_count.columns')
# print (clusters_count.values,'----clusters_count.counts')
# print ('bbbbbbb')
# print (clusters_count[0],'---clusters_count',clusters_count[1])
# 绘图展示
fig = plt.figure(figsize=(8,8))#建立画布
ax = plt.subplot(111) 
plt.pie(clusters_count.values[0],
       labels = clusters_count.columns,
       colors = ['darkcyan','c','lightcyan','aliceblue'],
       autopct='%.2f%%',
       radius=1)

In [ ]:
# 计算每个聚类类别内部最显著特征值
cluster_features = [] # 空列表，用于存储最终合并后的所有特征信息
for line in range(best_k): #读取每个类索引
    label_data = merge_data[merge_data['clusters'] == line] # 获得特定类的数据
    
    part1_data = label_data.iloc[:,1:7] # 获得数值型数据特征
    part1_desc = part1_data.describe().round(3) # # 得到数值型特征的描述性统计信息
    merge_data1 = part1_desc.iloc[2,:]  # 得到数值型特征的均值
    
    part2_data = label_data.iloc[:, 7:-1]  # 获得字符串型数据特征
    part2_desc = part2_data.describe(include='all')  # 获得字符串型数据特征的描述性统计信息
    merge_data2 = part2_desc.iloc[2, :]  # 获得字符串型数据特征的最频繁值
    
    merge_line = pd.concat((merge_data1,merge_data2),axis=0)# 将数值型和字符串型典型特征沿列合并
    cluster_features.append(merge_line) # 将每个类别下的数据特征追加到列表
    
# 输出完整的类别特征信息
clusters_pd = pd.DataFrame(cluster_features).T # 将列表转化为DataFrame

# 将每个聚类类别的所有信息合并
clusters_all = pd.concat((clusters_count,clusters_rat,clusters_pd),axis = 0)
print('每个类别的主要特征为：\n')
clusters_all

# 聚类结果可视化

In [ ]:
# 各类别数据预处理
num_sets = clusters_pd.iloc[:6,:].T.astype(np.float64) #获取展示数据
num_sets_max_min = model_norm.fit_transform(num_sets) #获得标准化后的数据
print (num_sets)
print (num_sets.shape)
# 绘制雷达图
fig = plt.figure(figsize=(8,8))#建立画布
ax = plt.subplot(111,polar = True) # 增加子网格，设置polar参数绘制极坐标
labels = np.array(merge_data1.index)  #设置要展示的数值数据标签
print (labels,'---labels')
cor_list = ['r','g','b','c'] # 定义不同类别的颜色
angles = np.linspace(0,2 * np.pi,len(labels),endpoint = False) # 计算各个区间的角度
angles = np.concatenate((angles,[angles[0]])) #建立相同首尾字段以便于闭合
for i in range(len(num_sets)): #循环每个类别
    print (i,'----')
    data_tmp = num_sets_max_min[i,:] #获得对应类数据
    data = np.concatenate((data_tmp,[data_tmp[0]])) # 建立相同首尾字段以便于闭合
    ax.plot(angles,data,'o-',c=cor_list[i],label = '渠道类别%i'%(i+1))# 画线
    ax.fill(angles,data,alpha = 0.5) # 区域填充颜色

#设置图像显示格式 set_thetagrids  set_xticklables xticks
print (labels,'----lables')
print (num_sets,'----num_sets',len(num_sets))
# ax.set_thetagrids(angles * 180 / np.pi,labels,fontproperties = 'Songti SC')# 设置极坐标轴
ax.set_title("各聚类类别显著特征对比", fontproperties="Songti SC")  # 设置标题放置
ax.set_rlim(-0.2,1.2) #设置坐标轴尺度范围
plt.legend(loc = 'upper right',bbox_to_anchor = (1.2,1.0))  #设置图例位置